# Part 1

Before you leave, the Elves in accounting just need you to fix your expense report (your puzzle input); apparently, something isn't quite adding up.

Specifically, they need you to find the two entries that sum to 2020 and then multiply those two numbers together.

For example, suppose your expense report contained the following:

```
1721
979
366
299
675
1456
```

In this list, the two entries that sum to 2020 are 1721 and 299. Multiplying them together produces `1721 * 299 = 514579`, so the correct answer is 514579.

Of course, your expense report is much larger. Find the two entries that sum to 2020; what do you get if you multiply them together?

In [1]:
from itertools import combinations
from pathlib import Path

In [2]:
INPUT_FILE = Path.cwd() / 'inputs' / 'day01' / 'part1.txt'

In [3]:
def input_numbers():
    with INPUT_FILE.open() as fp:
        for line in fp:
            yield int(line.strip())

In [4]:
def sum_to_2020(numbers):
    for n1, n2 in combinations(numbers, 2):
        if n1 + n2 == 2020:
            return n1, n2

In [5]:
n1, n2 = sum_to_2020(input_numbers())

In [6]:
n1, n2, n1 * n2

(1902, 118, 224436)

# Part 2

The Elves in accounting are thankful for your help; one of them even offers you a starfish coin they had left over from a past vacation. They offer you a second one if you can find three numbers in your expense report that meet the same criteria.

Using the above example again, the three entries that sum to 2020 are 979, 366, and 675. Multiplying them together produces the answer, 241861950.

In your expense report, what is the product of the three entries that sum to 2020?

In [7]:
def sum_to_2020_3(numbers):
    for n1, n2, n3 in combinations(numbers, 3):
        if n1 + n2 + n3 == 2020:
            return n1, n2, n3

In [8]:
n1, n2, n3 = sum_to_2020_3(input_numbers())

In [9]:
n1, n2, n3, n1 * n2 * n3

(689, 615, 716, 303394260)

# Addendum

This is me being silly and deciding to write my own `combinations` function.

In [10]:
from typing import Iterable

In [11]:
tuple(combinations('abcd', 1)), tuple(combinations('abcd', 2))

((('a',), ('b',), ('c',), ('d',)),
 (('a', 'b'), ('a', 'c'), ('a', 'd'), ('b', 'c'), ('b', 'd'), ('c', 'd')))

In [12]:
def my_combos(seq: Iterable, num: int) -> Iterable[tuple]:
    if num == 0:
        yield tuple()
    else:
        seq = tuple(seq)
        for index, item in enumerate(seq):
            for combos in my_combos(seq[index + 1:], num - 1):
                yield (item,) + combos

In [13]:
tuple(my_combos('abcd', 1)), tuple(my_combos('abcd', 2)), tuple(my_combos('abcd', 3))

((('a',), ('b',), ('c',), ('d',)),
 (('a', 'b'), ('a', 'c'), ('a', 'd'), ('b', 'c'), ('b', 'd'), ('c', 'd')),
 (('a', 'b', 'c'), ('a', 'b', 'd'), ('a', 'c', 'd'), ('b', 'c', 'd')))

Okay, but what if we don't like copying the tuples in there?

In [14]:
from itertools import islice

In [15]:
class ZeroCopy:
    def __init__(self, seq: tuple, start: int) -> None:
        self.seq = seq
        self.start = start
    
    def __iter__(self) -> Iterable:
        yield from islice(self.seq, self.start, None)
    
    def __getitem__(self, slice_: slice) -> 'ZeroCopy':
        if not isinstance(slice_, slice):
            raise ValueError('__getitem__ only supports slices')
        
        return ZeroCopy(self.seq, self.start + slice_.start)

In [16]:
def _my_combos2(seq: ZeroCopy, num: int) -> Iterable[tuple]:
    if num == 0:
        yield tuple()
    else:
        for index, item in enumerate(seq):
            for combos in _my_combos2(seq[index + 1:], num - 1):
                yield (item,) + combos

def my_combos2(seq: Iterable, num: int) -> Iterable[tuple]:
    yield from _my_combos2(ZeroCopy(tuple(seq), 0), num)

In [17]:
tuple(my_combos2('abcd', 1)), tuple(my_combos2('abcd', 2)), tuple(my_combos2('abcd', 3))

((('a',), ('b',), ('c',), ('d',)),
 (('a', 'b'), ('a', 'c'), ('a', 'd'), ('b', 'c'), ('b', 'd'), ('c', 'd')),
 (('a', 'b', 'c'), ('a', 'b', 'd'), ('a', 'c', 'd'), ('b', 'c', 'd')))

In [18]:
tuple(my_combos2('abcd', 5))

()

Ok but what if we don't want to be concatenating tuples in there?

In [19]:
from typing import Any, Optional

In [20]:
class LinkedItem:
    def __init__(self, item: Any, next_: Optional['LinkedItem'] = None) -> None:
        self.item = item
        self.next_ = next_
    
    def __iter__(self):
        yield self.item
        if self.next_:
            yield from iter(self.next_)

In [21]:
def _my_combos3(seq: ZeroCopy, num: int) -> Iterable[tuple]:
    if num == 0:
        yield None
    else:
        for index, item in enumerate(seq):
            for combos in _my_combos3(seq[index + 1:], num - 1):
                yield LinkedItem(item, combos)

def my_combos3(seq: Iterable, num: int) -> Iterable[tuple]:
    yield from (tuple(combo) for combo in _my_combos2(ZeroCopy(tuple(seq), 0), num))

In [22]:
tuple(my_combos3('abcd', 1)), tuple(my_combos3('abcd', 2)), tuple(my_combos3('abcd', 3))

((('a',), ('b',), ('c',), ('d',)),
 (('a', 'b'), ('a', 'c'), ('a', 'd'), ('b', 'c'), ('b', 'd'), ('c', 'd')),
 (('a', 'b', 'c'), ('a', 'b', 'd'), ('a', 'c', 'd'), ('b', 'c', 'd')))

In [23]:
tuple(my_combos3('abcd', 5))

()

In [24]:
%timeit tuple(combinations('abcdefg', 4))

1.4 µs ± 4.44 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [25]:
%timeit tuple(my_combos('abcdefg', 4))

60.2 µs ± 448 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [26]:
%timeit tuple(my_combos2('abcdefg', 4))

138 µs ± 722 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [27]:
%timeit tuple(my_combos3('abcdefg', 4))

143 µs ± 2.11 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
